# Make figures

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import glob
import sys
from shapely import wkt
import seaborn as sns

In [ ]:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping-application/'
sys.path.append(os.path.join(base_path, 'functions'))
import model_analyze_utils as f

scm_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'
figures_out_path = os.path.join(base_path, 'figures')
aois_path = os.path.join(scm_path, 'all_AOIs')
aois_fn = 'all_aois.shp'
snowlines_path = os.path.join(scm_path, 'all_snowlines')
snowlines_fn = 'all_snowlines.csv'
snowlines_medians_fn = 'all_snowlines_weekly_median_trends.csv'

## Plot minimum AAR with different characteristics

In [ ]:
# -----Load minimum snow cover characteristics
min_snow_cover_stats_fn = 'min_snow_cover_stats.csv'
min_snow_cover_stats = pd.read_csv(os.path.join(snowlines_path, min_snow_cover_stats_fn))
min_snow_cover_stats['geometry'] = min_snow_cover_stats['geometry'].apply(wkt.loads)
# sort by region numbers
min_snow_cover_stats.sort_values(by=['O1Region', 'O2Region'], inplace=True)

# -----Add glacier boundary x and y centroid coordinates for plotting
min_snow_cover_stats['centroid_x'] = [x.centroid.coords.xy[0][0] for x in min_snow_cover_stats['geometry']]
min_snow_cover_stats['centroid_y'] = [x.centroid.coords.xy[1][0] for x in min_snow_cover_stats['geometry']]

# -----Load country outlines for plotting
countries_fn = os.path.join(scm_path, '..', 'GIS_data', 'countries_shp', 'countries.shp')
countries = gpd.read_file(countries_fn)
usca = countries.loc[(countries['NAME']=='United States') | (countries['NAME']=='Canada')].reset_index(drop=True)

In [ ]:
# -----Plot maps of minimum AARs by various columns
columns = ['Subregion', 'Aspect', 'Zmed', 'Area', 'Slope']
columns_display = ['Subregion', 'Aspect [degrees]', 'Median elevation [m]', 'Area [km$^2$]', 'Slope [degrees]']
palettes = [dict(min_snow_cover_stats[['Subregion', 'color']].drop_duplicates().values),
            'hls', 
            'mako', 
            'viridis', 
            'crest']
bins_list = ['N/A', 
             np.linspace(0,360, num=7), 
             np.linspace(500, 3000, num=6),
             np.linspace(0, 1500, num=7),
             np.linspace(0, 35, num=8)]
plt.rcParams.update({'font.sans-serif':'Arial', 'font.size':12})
xmin, xmax = -168, -112
ymin, ymax = 45, 65
for column, column_display, palette, bins in zip(columns, columns_display, palettes, bins_list):
    fig, ax = plt.subplots(2, 1, figsize=(10,10), gridspec_kw={'height_ratios':[2.5, 1]})
    # plot country outlines on the map
    usca.plot(ax=ax[0], facecolor='None', edgecolor='grey')
    # plot points on the map
    sns.scatterplot(data=min_snow_cover_stats, x='centroid_x', y='centroid_y', 
                    hue=column, palette=palette, size='AAR_P50_min', sizes=(5, 100), legend=True, ax=ax[0])
    sns.move_legend(ax[0], "center right", bbox_to_anchor=[1.1, 0.4, 0.2, 0.2])
    ax[0].set_xlabel('')
    ax[0].set_ylabel('')
    ax[0].grid()
    ax[0].set_xlim(xmin, xmax)
    ax[0].set_ylim(ymin, ymax)
    # plot regional barplots
    if type(bins) != str:
        # add bin column
        min_snow_cover_stats[column + '_bin'] = pd.cut(min_snow_cover_stats[column], bins)
        # plot boxplots
        sns.boxplot(data=min_snow_cover_stats, x=column + '_bin', y='AAR_P50_min', showfliers=False, palette=palette, ax=ax[1])
    else:
        # plot boxplots
        sns.boxplot(data=min_snow_cover_stats, x=column, y='AAR_P50_min', showfliers=False, palette=palette, ax=ax[1])
    if type(bins) == str:
        ax[1].set_xticks(ax[1].get_xticks())
        ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
    ax[1].set_xlabel(column_display)
    ax[1].set_ylabel('AAR')

    # Edit legend labels for map
    handles, labels = ax[0].get_legend_handles_labels()
    labels = [x.replace('AAR_P50_min', 'Median AAR').replace(column, column_display) for x in labels]
    ax[0].legend(handles, labels, loc='center right')
    sns.move_legend(ax[0], "center right", bbox_to_anchor=[1.1, 0.4, 0.2, 0.2])
    
    plt.show()

    # Edit legend
    handles, labels = ax[0].get_legend_handles_labels()
    
    # Save figure
    fig_fn = 'minimum_AARs_map_' + column + '.png'
    fig.savefig(os.path.join(figures_out_path, fig_fn), dpi=250, bbox_inches='tight')
    print('figure saved to file: ', os.path.join(figures_out_path, fig_fn))

## Plot linear changes in AARs for 2016--2023 by terrain characteristics

In [ ]:
# -----Load AARs linear fit
aars_linear_fit_fn = 'minimum_AARs_linear_fit.csv'
aars_linear_fit = pd.read_csv(os.path.join(snowlines_path, aars_linear_fit_fn))

# -----Add glacier terrain and geometry columns
columns = ['geometry', 'centroid_x', 'centroid_y', 'O1Region', 'O2Region', 'Subregion', 'color', 'Aspect', 'Slope', 'Zmed', 'Area']
for column in columns:
    aars_linear_fit[column] = np.nan
# iterate over site names
for site_name in aars_linear_fit['site_name'].drop_duplicates().values:
    min_snow_cover_stats_site = min_snow_cover_stats.loc[min_snow_cover_stats['RGIId']==site_name]
    try:
        for column in columns:
            aars_linear_fit.loc[aars_linear_fit['site_name']==site_name, column] = [min_snow_cover_stats_site[column].values[0]]
    except:
        continue

aars_linear_fit.dropna(inplace=True)

# -----Add column for negative value of change category to show more negative values as larger markers
aars_linear_fit['linear_fit_coef_neg'] = - aars_linear_fit['linear_fit_coef']
# Sort by region numbers
aars_linear_fit.sort_values(by=['O1Region', 'O2Region'], inplace=True)
aars_linear_fit

In [ ]:
# -----Plot map of AARs linear fit with various columns
columns = ['Subregion', 'Aspect', 'Zmed', 'Area', 'Slope']
columns_display = ['Subregion', 'Aspect [degrees]', 'Median elevation [m]', 'Area [km$^2$]', 'Slope [degrees]']
palettes = [dict(aars_linear_fit[['Subregion', 'color']].drop_duplicates().values),
            'hls', 
            'mako', 
            'viridis', 
            'crest']
bins_list = ['N/A', 
             np.linspace(0,360, num=7), 
             np.linspace(500, 3000, num=6),
             np.linspace(0, 1500, num=7),
             np.linspace(0, 35, num=8)]
plt.rcParams.update({'font.sans-serif':'Arial', 'font.size':12})
xmin, xmax = -168, -112
ymin, ymax = 45, 65
def map_size(value):
    return np.abs(value) * 100
for column, column_display, palette, bins in zip(columns, columns_display, palettes, bins_list):
    fig, ax = plt.subplots(2, 1, figsize=(10,10), gridspec_kw={'height_ratios':[2.5, 1]})
    # plot country outlines on the map
    usca.plot(ax=ax[0], facecolor='None', edgecolor='grey')
    # plot points on the map
    sns.scatterplot(data=aars_linear_fit, x='centroid_x', y='centroid_y', 
                    hue=column, palette=palette, size='linear_fit_coef_neg', sizes=(5, 150), legend=True, ax=ax[0])
    ax[0].set_xlabel('')
    ax[0].set_ylabel('')
    ax[0].grid()
    ax[0].set_xlim(xmin, xmax)
    ax[0].set_ylim(ymin, ymax)
    # plot regional barplots
    if type(bins) != str:
        # add bin column
        aars_linear_fit[column + '_bin'] = pd.cut(aars_linear_fit[column], bins)
        # plot boxplots
        sns.boxplot(data=aars_linear_fit, x=column + '_bin', y='linear_fit_coef', showfliers=False, palette=palette, ax=ax[1])
    else:
        # plot boxplots
        sns.boxplot(data=aars_linear_fit, x=column, y='linear_fit_coef', showfliers=False, palette=palette, ax=ax[1])
    if type(bins) == str:
        ax[1].set_xticks(ax[1].get_xticks())
        ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
    ax[1].set_xlabel(column_display)
    ax[1].set_ylabel('AAR linear trend [per year]')
    
    # Edit legend labels for map
    handles, labels = ax[0].get_legend_handles_labels()
    labels = [x.replace('0.', '-0.').replace('linear_fit_coef_neg', 'AAR linear trend [1/year]').replace(column, column_display) for x in labels]
    labels = [x.replace('-0.0', '0.0') for x in labels]
    ax[0].legend(handles, labels, loc='center right')
    sns.move_legend(ax[0], "center right", bbox_to_anchor=[1.15, 0.4, 0.2, 0.2])

    plt.show()

    # Save figure
    fig_fn = 'AARs_linear_fit_map_' + column + '.png'
    fig.savefig(os.path.join(figures_out_path, fig_fn), dpi=250, bbox_inches='tight')
    print('figure saved to file: ', os.path.join(figures_out_path, fig_fn))